# Prep Data

In [8]:
import pandas as pd

df = (pd
        .read_csv('https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years-violent.csv')
        .drop(columns=['two_year_recid.1','violent_recid'])
     )

y = df.apply(lambda row: 'No-Recidivism' if row['is_recid'] == 0 
                    else 'Violent' if row['is_violent_recid']==0
                    else 'Non-Violent', axis=1
            )

X = (df.drop(columns=['is_recid','is_violent_recid', 'two_year_recid',
                      'decile_score','decile_score.1','v_decile_score','score_text','v_score_text',
                      'compas_screening_date','c_days_from_compas','c_case_number','priors_count.1',
                      'r_charge_desc', 'vr_charge_desc', 'vr_charge_degree', 'r_charge_degree',
                      'vr_offense_date','r_offense_date', 'event', 'end',
                      'r_days_from_arrest', 'r_case_number','vr_case_number',
                         ])
     
    .rename(columns = {'dob':'date_of_birth','age_cat':'age_group', })
)

groups = (X
          [['age_group', 'sex', 'race']]
          .apply(lambda x: tuple(x), axis=1)
          .astype('str')
          .pipe(lambda x: x.replace(list(x.value_counts().tail(3).to_dict().keys()), 'Other'))
         )

In [9]:
from sklearn.model_selection import train_test_split

#!mkdir train
#!mkdir test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=groups, random_state=1)

In [10]:
X_train.to_csv('train/X_train.csv', index=False)
X_test.to_csv('test/X_test.csv', index=False)

y_train.to_csv('train/y_train.csv', index=False)
y_test.to_csv('test/y_test.csv', index=False)